In [1]:
%pwd

u'C:\\Users\\Aravindh\\aws-ec2'

In [2]:
import os, sys
current_dir = os.getcwd()
HOME = os.path.join('c:/','Users','Aravindh')
#HOME = '/home/ubuntu'
DATA_HOME_DIR= os.path.join(HOME,'data','redux')
sys.path.append(os.path.expanduser('current_dir+/utils/'))

In [3]:
DATA_HOME_DIR

'c:/Users\\Aravindh\\data\\redux'

In [4]:
from Vgg import *
import cv2
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
%matplotlib inline

Using Theano backend.


## Action Plan

1. Create Validation and Sample sets
2. Rearrange image files into their respective directories
3. Finetune and Train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

### Create validation and sample sets

In [37]:
# create directories
os.chdir(DATA_HOME_DIR)
if not os.path.exists('valid'): os.makedirs('valid')
if not os.path.exists('results'): os.makedirs('results')
if not os.path.exists(os.path.join('sample','train')): os.makedirs(os.path.join('sample','train'))
if not os.path.exists(os.path.join('sample','test')): os.makedirs(os.path.join('sample','test'))
if not os.path.exists(os.path.join('sample','valid')): os.makedirs(os.path.join('sample','valid'))
if not os.path.exists(os.path.join('sample','results')): os.makedirs(os.path.join('sample','results'))

In [57]:
# move a subset of the train data to validation directory
os.chdir(os.path.join(DATA_HOME_DIR,'train'))
g = glob('*.jpg')
shuf = np.random.permutation(g) # create a random permutation of cats and dogs
for i in range(2000): os.rename(shuf[i],os.path.join(DATA_HOME_DIR,'valid',shuf[i]))

In [58]:
# create a sample training set
os.chdir(os.path.join(DATA_HOME_DIR,'train'))
g = glob('*.jpg')
shuf = np.random.permutation(g) # create a random permutation of cats and dogs
for i in range(200): os.rename(shuf[i],os.path.join(DATA_HOME_DIR,'sample','train',shuf[i]))

In [59]:
# create a sample vaidation set
os.chdir(os.path.join(DATA_HOME_DIR,'train'))
g = glob('*.jpg')
shuf = np.random.permutation(g) # create a random permutation of cats and dogs
for i in range(50): os.rename(shuf[i],os.path.join(DATA_HOME_DIR,'sample','valid',shuf[i]))

### Rearrange image files in to their respective directories

In [62]:
# sample/train
os.chdir(os.path.join(DATA_HOME_DIR,'sample','train'))
if not os.path.exists('cats'): os.makedirs('cats')
if not os.path.exists('dogs'): os.makedirs('dogs')
for img in glob('cat*.jpg'): os.rename(img,os.path.join('cats',img))
for img in glob('dog*.jpg'): os.rename(img,os.path.join('dogs',img))

In [63]:
# sample/valid
os.chdir(os.path.join(DATA_HOME_DIR,'sample','valid'))
if not os.path.exists('cats'): os.makedirs('cats')
if not os.path.exists('dogs'): os.makedirs('dogs')
for img in glob('cat*.jpg'): os.rename(img,os.path.join('cats',img))
for img in glob('dog*.jpg'): os.rename(img,os.path.join('dogs',img))

In [64]:
# valid
os.chdir(os.path.join(DATA_HOME_DIR,'valid'))
if not os.path.exists('cats'): os.makedirs('cats')
if not os.path.exists('dogs'): os.makedirs('dogs')
for img in glob('cat*.jpg'): os.rename(img,os.path.join('cats',img))
for img in glob('dog*.jpg'): os.rename(img,os.path.join('dogs',img))

In [65]:
# train
os.chdir(os.path.join(DATA_HOME_DIR,'train'))
if not os.path.exists('cats'): os.makedirs('cats')
if not os.path.exists('dogs'): os.makedirs('dogs')
for img in glob('cat*.jpg'): os.rename(img,os.path.join('cats',img))
for img in glob('dog*.jpg'): os.rename(img,os.path.join('dogs',img))

### Finetuning and training

In [5]:
path = DATA_HOME_DIR + '/sample'
#path = os.path.join(DATA_HOME_DIR)
test_path = os.path.join(DATA_HOME_DIR,'test') #We use all the test data
results_path = os.path.join(DATA_HOME_DIR,'results')
train_path = os.path.join(path,'train')
valid_path = os.path.join(path,'valid')

In [6]:
# create model, load weights
model = Vgg()
model.load_weights(HOME+'/models/vgg16.h5')
model.load_imagenet_class_index(HOME+'/models/imagenet_class_index.json')

Hello from Vgg()


In [7]:
# Set batches and epocs
batch_size = 64
number_of_epochs = 3

In [8]:
# get and preprocess training images
train_batches = model.get_batches(train_path,batch_size=batch_size)
val_batches = model.get_batches(valid_path,batch_size=batch_size*2)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [9]:
# fine tune model to output two classes instead to vgg's 1000
model.finetune(train_batches.nb_class)

In [ ]:
# Train model
for epochs in range(number_of_epochs): model.fit(train_batches, val_batches, nb_epoch=1)

Epoch 1/1
128/200 [==================>...........] - ETA: 510s - loss: 1.0288 - acc: 0.6719